<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Evaluate_Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this notebook we will evaluate the results of the previously generated summaries with different both extractive and abstractive models. For this we will use both manually implemented and other methods like BLANC

In [1]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)
!pip install -U sentence-transformers
!pip install blanc

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 507 (delta 100), reused 100 (delta 87), pack-reused 378
Receiving objects: 100% (507/507), 30.39 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (236/236), done.
Checking out files: 100% (127/127), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=36dd29e1b766f3b876ffdd220330326dba5f0d27f1f0a74709d709fd0362bf17
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Loo

In [2]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
df = pd.read_csv('/content/summaries (1).csv')
df.head()

,Unnamed: 0,name,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,0,news,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,"The Chrysler Building, the famous art deco New...","The Chrysler Building, the famous art deco New..."
1,1,allhealth,Dr. Karen Bell is the chair of the Certificat...,Dr. Karen Bell is the chair of the Certificati...,The promise of Accountable Care is tempered by...,Dr. Karen Bell is the chair of the Certificati...,Dr. Karen Bell is the chair of the Certificati...
2,2,health_today,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...
3,3,rfl_1,"All right, so here is a rule we're going to c...","The TD (0) rule looks like this, not so unfami...",The TD (0) rule is used to calculate the maxim...,"All right, so here is a rule we're going to ca...","All right, so here is a rule we're going to ca..."


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity

def evaluate(model, paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_embeddings = model.encode(sent_tokenize(paragraph))

  # Get sentence embedding for summary
  summary_embeddings = model.encode(sent_tokenize(summary))

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_embeddings:
    max = 0
    for paragraph in paragraph_embeddings:
      similarity = cosine_similarity(summary, paragraph) 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [6]:
from blanc import BlancHelp, BlancTune

# blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)
blanc_help = BlancHelp(device='cuda')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
metrics = pd.DataFrame()

### Abstractive

In [8]:
metrics['bart_score'] = df[['content', 'bart']].apply(lambda x: evaluate(model, x['content'], x['bart']), axis=1)
metrics['pegasus_score'] = df[['content', 'pegasus']].apply(lambda x: evaluate(model, x['content'], x['pegasus']), axis=1)

In [9]:
df['pegasus_blanc'] = df[['content', 'pegasus']].apply(lambda x: blanc_help.eval_once(x['content'], x['pegasus']), axis=1)
df['bart_blanc'] = df[['content', 'bart']].apply(lambda x: blanc_help.eval_once(x['content'], x['bart']), axis=1)

100%|██████████| 186/186 [00:02<00:00, 62.31it/s]


### Extractive

In [10]:
metrics['bert_extractive_blanc'] = df[['content', 'bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['bert_extractive']), axis=1)
metrics['dist_bert_extractive_blanc'] = df[['content', 'dist_bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['dist_bert_extractive']), axis=1)

100%|██████████| 186/186 [00:03<00:00, 46.67it/s]


In [8]:
# metrics['bert_extractive_score'] = df[['content', 'bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['bert_extractive']), axis=1)
# metrics['dist_bert_extractive_score'] = df[['content', 'dist_bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['dist_bert_extractive']), axis=1)

In [11]:
metrics.head()

,bart_score,pegasus_score,bert_extractive_blanc,dist_bert_extractive_blanc
0,0.965930,0.810819,0.165493,0.169014
1,0.908323,0.868556,0.203004,0.189539
2,0.825120,0.787513,0.250633,0.250633
3,0.828703,0.855091,0.182609,0.252174


In [ ]:
metrics.head()

,bart_score,pegasus_score,bert_extractive_score,dist_bert_extractive_score
0,0.965930,0.810819,1.0,1.0
1,0.908323,0.868556,1.0,1.0
2,0.825120,0.787513,1.0,1.0
3,0.828703,0.855091,1.0,1.0


In [ ]:
print('BART score ', metrics['bart_score'].mean())
print('Pegasus score', metrics['pegasus_score'].mean())

BART score  0.8820190027001359
Pegasus score 0.8304945679776595
